Name: Muhammed Selim Alpa

ID: 161101042

Course: YAP476 /BIL573

In [86]:
import pandas as pd
import os

In [100]:
path = "C:/Users/selim/OneDrive/Masaüstü/RetailSalesData"
product_hierarchy_path = path+'/product.csv'
store_cities_path = path+'/store.csv'
sales_csv_path = path+'/sales.csv'

### TASK-1: Load Data

In [88]:

product_df = pd.read_csv(product_hierarchy_path)
store_df = pd.read_csv(store_cities_path)
sales_df = pd.read_csv(sales_csv_path)


### TASK-2: Design and Implement Star Schema

Design the Star Schema
A typical star schema consists of a central fact table surrounded by dimension tables. 

Fact Table:
1. sales_fact: Stores sales data with foreign keys linking to dimension tables.
2. Measures: price, quantity_sold, stock.

Dimension Tables:
1. product_dim: Stores product details. 
2. store_dim: Stores store details.
3. time_dim: Stores date and time details.('date_id', 'date (text)', 'year', 'month', 'day') 

For time_dim:
1. Drop dublicate dates from sales_df
2. Convert the date column  to datetime; use pd.to_datetime() to convert the date column to a datetime object.
3. Extract date components for creating new columns for year, month, day, and keep the original date column as text and add index as date_id.
4. Insert data to time_dim table

In [89]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS sales_fact')
cursor.execute('DROP TABLE IF EXISTS product_dim')
cursor.execute('DROP TABLE IF EXISTS store_dim')
cursor.execute('DROP TABLE IF EXISTS time_dim')

conn.commit()
print(store_df.columns)
print(product_df.columns)
print(sales_df.columns)


# Fact Table: sales_fact
cursor.execute('''
    CREATE TABLE sales_fact (
        sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        store_id INTEGER,
        date_id INTEGER,
        price DOUBLE,
        quantity_sold INTEGER,
        stock INTEGER,
        FOREIGN KEY (product_id) REFERENCES product_dim(product_id),
        FOREIGN KEY (store_id) REFERENCES store_dim(store_id),
        FOREIGN KEY (date_id) REFERENCES time_dim(date_id)
    )
''')

# Dimension Table: product_dim
cursor.execute('''
    CREATE TABLE product_dim (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        sub_category TEXT
    )
''')

# Dimension Table: store_dim
cursor.execute('''
    CREATE TABLE store_dim (
        store_id INTEGER PRIMARY KEY,
        store_name TEXT,
        city_id TEXT,
        store_size INTEGER
    )
''')

# Dimension Table: time_dim
cursor.execute('''
    CREATE TABLE time_dim (
        date_id INTEGER PRIMARY KEY,
        date TEXT,
        year INTEGER,
        month INTEGER,
        day INTEGER
    )
''')

# Değişiklikleri kaydet
conn.commit()


# Insert data into product_dim
product_df.to_sql('product_dim', conn, if_exists='append', index=False)

# Insert data into store_dim
store_df_filtered = store_df[['store_id', 'store_size', 'city_id', 'store_name']]
store_df_filtered.to_sql('store_dim', conn, if_exists='append', index=False)

 # Insert data into time_dim
# Tarih kolonunu datetime formatına çevirme
sales_df['date'] = pd.to_datetime(sales_df['date'])

# time_dim tablosuna veri ekleme
time_dim_df = sales_df[['date']].drop_duplicates().reset_index(drop=True)
time_dim_df['year'] = time_dim_df['date'].dt.year
time_dim_df['month'] = time_dim_df['date'].dt.month
time_dim_df['day'] = time_dim_df['date'].dt.day
time_dim_df['date_id'] = time_dim_df['date'].dt.strftime('%Y%m%d').astype(int)

time_dim_df.to_sql('time_dim', conn, if_exists='append', index=False)

# Insert data into sales_fact
# time_dim tablosunu satışlar tablosuna ekleme
sales_fact_df = sales_df.merge(time_dim_df, on='date')
sales_fact_df = sales_fact_df[['product_id', 'store_id', 'date_id', 'sales', 'price', 'stock']]
sales_fact_df.rename(columns={'sales': 'quantity_sold'}, inplace=True)
sales_fact_df.to_sql('sales_fact', conn, if_exists='append', index=False)

# Değişiklikleri kaydet ve bağlantıyı kapat
conn.commit()
conn.close()

Index(['store_id', 'storetype_id', 'store_size', 'city_id', 'store_name'], dtype='object')
Index(['product_id', 'product_name', 'category', 'sub_category'], dtype='object')
Index(['sale_id', 'product_id', 'store_id', 'date', 'sales', 'price', 'stock'], dtype='object')


Verify the Data

 Connect to database and display all  dimension tables and fact table, you can  show only first 5 input of tables

In [90]:
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

# Product Dimension tablosunu kontrol et ve ilk 5 satırı göster
print("Product Dimension:")
product_dim = pd.read_sql_query("SELECT * FROM product_dim LIMIT 5;", conn)
display(product_dim)

# Store Dimension tablosunu kontrol et ve ilk 5 satırı göster
print("\nStore Dimension:")
store_dim = pd.read_sql_query("SELECT * FROM store_dim LIMIT 5;", conn)
display(store_dim)

# Time Dimension tablosunu kontrol et ve ilk 5 satırı göster
print("\nTime Dimension:")
time_dim = pd.read_sql_query("SELECT * FROM time_dim LIMIT 5;", conn)
display(time_dim)

# Sales Fact tablosunu kontrol et ve ilk 5 satırı göster
print("\nSales Fact:")
sales_fact = pd.read_sql_query("SELECT * FROM sales_fact LIMIT 400;", conn)
display(sales_fact)

# Bağlantıyı kapat
conn.close()

Product Dimension:


,product_id,product_name,category,sub_category
0,0,P0000,H00,H0004
1,1,P0001,H01,H0105
2,2,P0002,H03,H0315
3,3,P0004,H03,H0314
4,4,P0005,H03,H0312



Store Dimension:


,store_id,store_name,city_id,store_size
0,0,S0091,C013,19
1,1,S0012,C005,28
2,2,S0045,C008,17
3,3,S0032,C019,14
4,4,S0027,C022,24



Time Dimension:


,date_id,date,year,month,day
0,20170102,2017-01-02 00:00:00,2017,1,2
1,20170103,2017-01-03 00:00:00,2017,1,3
2,20170104,2017-01-04 00:00:00,2017,1,4
3,20170105,2017-01-05 00:00:00,2017,1,5
4,20170106,2017-01-06 00:00:00,2017,1,6



Sales Fact:


,sale_id,product_id,store_id,date_id,price,quantity_sold,stock
0,1,1,67,20170102,6.25,0,8
1,2,1,1,20170102,6.25,1,0
2,3,1,139,20170102,6.25,2,0
3,4,1,94,20170102,6.25,0,6
4,5,1,108,20170102,6.25,0,1
...,...,...,...,...,...,...,...
395,396,1,134,20170102,2.45,2,10
396,397,1,124,20170102,2.45,0,1
397,398,1,101,20170102,2.45,0,11
398,399,1,98,20170102,2.45,0,23


### TASK-3: Perform OLAP Operations

In [101]:
import sqlite3
import pandas as pd
path = "C:/Users/selim/OneDrive/Masaüstü/RetailSalesData"


# Reconnect to the database
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

Example Queries:

Roll-up: Aggregates data along a dimension. For example, aggregate stocks by month.

Drill-down: Opposite of roll-up. For example, break down sales(quantity_sold) by day.

Slice: Select a single dimension of data. For example, sales(quantity_sold) for a specific product (P0006).

Dice: Select multiple dimensions of data. For example, sales(quantity_sold) for specific products ('P0006','P00017') and stores ('S0091', 'S0027')

Pivot: Reorganize data along different dimensions. For example, pivot sales data by store and product.

Within the scope of this task, you will implement the OLAP operations given above as examples, in addition, you will define 1 example for each type of OLAP operation and you will implement them as well.

In [92]:
# Roll-up: Aggregate sales by month

rollup_query = '''
SELECT strftime('%Y', t.date) AS year, strftime('%m', t.date) AS month, SUM(s.quantity_sold) AS total_stock
FROM sales_fact s
JOIN time_dim t ON s.date_id = t.date_id
GROUP BY year, month;
'''
rollup_df = pd.read_sql_query(rollup_query, conn)
print("Roll-up (Aggregate sales by month):")
display(rollup_df)

# Drill-down: Break down sales by day
drilldown_query = '''
SELECT strftime('%Y', t.date) AS year, strftime('%m', t.date) AS month, strftime('%d', t.date) AS day, SUM(s.quantity_sold) AS total_sales
FROM sales_fact s
JOIN time_dim t ON s.date_id = t.date_id
GROUP BY t.date;
'''
drilldown_df = pd.read_sql_query(drilldown_query, conn)
print("\nDrill-down (Break down sales by day):")
display(drilldown_df)

# Slice: Sales for a specific product (P0006)
slice_query = '''
SELECT strftime('%Y', t.date) AS year, strftime('%m', t.date) AS month, SUM(s.quantity_sold) AS total_sales
FROM sales_fact s
JOIN time_dim t ON s.date_id = t.date_id
JOIN product_dim p ON s.product_id = p.product_id
WHERE p.product_name = 'P0006'
GROUP BY year, month;
'''
slice_df = pd.read_sql_query(slice_query, conn)
print("\nSlice (Sales for a specific product):")
display(slice_df)

# Dice: Sales (quantity_sold) for specific products ('P0006','P00017') and stores ('S0091', 'S0027')
dice_query = '''
SELECT strftime('%Y', t.date) AS year, strftime('%m', t.date) AS month, 
           p.product_name, st.store_name, SUM(s.quantity_sold) AS total_sales
    FROM sales_fact s
    JOIN time_dim t ON s.date_id = t.date_id
    JOIN product_dim p ON s.product_id = p.product_id
    JOIN store_dim st ON s.store_id = st.store_id
    WHERE p.product_name IN ('P0006', 'P00017')
      AND st.store_name IN ('S0027', 'S0091')
    GROUP BY year, month, p.product_name, st.store_name;
'''
dice_df = pd.read_sql_query(dice_query, conn)
print("\nDice (Sales for specific products and stores):")
display(dice_df)

# Pivot: Sales by store and product
pivot_query = '''
SELECT st.store_name, p.product_name, SUM(s.quantity_sold) AS total_sales
    FROM sales_fact s
    JOIN product_dim p ON s.product_id = p.product_id
    JOIN store_dim st ON s.store_id = st.store_id
    GROUP BY st.store_name, p.product_name;
'''
pivot_df = pd.read_sql_query(pivot_query, conn)
print("\nPivot (Sales by store and product):")
display(pivot_df)

# Close the connection
conn.close()




Roll-up (Aggregate sales by month):


,year,month,total_stock
0,2017,01,227749.584
1,2017,02,291119.054
2,2017,03,232344.429
3,2017,04,261522.274
4,2017,05,214037.844
5,2017,06,208024.868
6,2017,07,227654.857
7,2017,08,239152.442
8,2017,09,224433.537
9,2017,10,215012.960



Drill-down (Break down sales by day):


,year,month,day,total_sales
0,2017,01,02,6817.962
1,2017,01,03,6578.220
2,2017,01,04,6615.686
3,2017,01,05,6440.778
4,2017,01,06,7182.927
...,...,...,...,...
1087,2019,12,25,NaN
1088,2019,12,26,NaN
1089,2019,12,27,NaN
1090,2019,12,28,NaN



Slice (Sales for a specific product):


,year,month,total_sales
0,2017,01,2872.611



Dice (Sales for specific products and stores):


,year,month,product_name,store_name,total_sales
0,2017,01,P0006,S0027,9
1,2017,01,P0006,S0091,14



Pivot (Sales by store and product):


,store_name,product_name,total_sales
0,S0001,P0001,134.00
1,S0001,P0002,8.00
2,S0001,P0004,20.00
3,S0001,P0005,199.51
4,S0001,P0006,14.51
...,...,...,...
85220,S0144,P0742,NaN
85221,S0144,P0745,NaN
85222,S0144,P0746,NaN
85223,S0144,P0747,NaN


### Task 4: Join Indexing and Performance Comparison

Measure the performance of the OLAP queries with and without indexes using Python's time module.

STEPS:

1. Create join indexes.

2. Measure the performance of the OLAP queries with the indexes.

3. Drop the indexes.

4. Measure the performance of the OLAP queries without the indexes.

4. Compare the performance and calculate the improvement.

Create an index that combines these columns to speed up the join operations.

Example: In the context of our data warehouse, we have a sales_fact table that joins with product_dim, store_dim, and time_dim. We can create join indexes on these columns.

These cresated indexes will help optimize queries that involve joining the sales_fact table with the product_dim, store_dim, and time_dim tables on the specified columns.


In [105]:
import sqlite3
import pandas as pd
import time

# Function to execute a SQL query and measure execution time
def execute_query(query, conn):
    start_time = time.time()
    df = pd.read_sql_query(query, conn)
    end_time = time.time()
    return df, end_time - start_time

# Reconnect to the database
conn = sqlite3.connect(path+'/sales_data_warehouse.db')
cursor = conn.cursor()

# Create join indexes (views)
create_index_queries = [
    '''
    CREATE VIEW sales_fact_with_index AS
    SELECT s.*, t.year, t.month
    FROM sales_fact s
    JOIN time_dim t ON s.date_id = t.date_id
    ''',
    '''
    CREATE VIEW sales_fact_with_index2 AS
    SELECT s.*, p.product_name
    FROM sales_fact s
    JOIN product_dim p ON s.product_id = p.product_id
    ''',
    '''
    CREATE VIEW sales_fact_with_index3 AS
    SELECT s.*, st.store_name
    FROM sales_fact s
    JOIN store_dim st ON s.store_id = st.store_id
    '''
]

# Execute index creation queries
for query in create_index_queries:
    cursor.execute(query)

# Commit changes
conn.commit()

# Define queries
queries = {
    'rollup': '''
        SELECT t.year, t.month, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN time_dim t ON s.date_id = t.date_id
        GROUP BY t.year, t.month
        ORDER BY t.year, t.month;
    ''',
    'drilldown': '''
        SELECT t.year, t.month, t.day, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN time_dim t ON s.date_id = t.date_id
        GROUP BY t.year, t.month, t.day
        ORDER BY t.year, t.month, t.day;
    ''',
    'slice': '''
        SELECT t.year, t.month, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN time_dim t ON s.date_id = t.date_id
        JOIN product_dim p ON s.product_id = p.product_id
        WHERE p.product_name = 'Specific Product'
        GROUP BY t.year, t.month
        ORDER BY t.year, t.month;
    ''',
    'dice': '''
        SELECT t.year, t.month, p.product_name, st.store_name, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN time_dim t ON s.date_id = t.date_id
        JOIN product_dim p ON s.product_id = p.product_id
        JOIN store_dim st ON s.store_id = st.store_id
        WHERE p.product_name IN ('Product A', 'Product B') AND st.store_name IN ('Store X', 'Store Y')
        GROUP BY t.year, t.month, p.product_name, st.store_name
        ORDER BY t.year, t.month, p.product_name, st.store_name;
    ''',
    'pivot': '''
        SELECT st.store_name, p.product_name, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN store_dim st ON s.store_id = st.store_id
        JOIN product_dim p ON s.product_id = p.product_id
        GROUP BY st.store_name, p.product_name
        ORDER BY st.store_name, p.product_name;
    '''
}

# Measure performance with indexes
performance_with_indexes = {}
for query_name, query_text in queries.items():
    df, exec_time = execute_query(query_text, conn)
    performance_with_indexes[query_name] = exec_time
    print(f"{query_name} with indexes: {exec_time:.6f} seconds")

print("\nPerformance with Indexes:")
print(performance_with_indexes)

# Drop indexes/views
drop_index_queries = [
    '''DROP VIEW IF EXISTS sales_fact_with_index''',
    '''DROP VIEW IF EXISTS sales_fact_with_index2''',
    '''DROP VIEW IF EXISTS sales_fact_with_index3'''
]

# Execute drop index queries
for query in drop_index_queries:
    cursor.execute(query)

# Commit changes
conn.commit()

# Measure performance without indexes
performance_without_indexes = {}
for query_name, query_text in queries.items():
    df, exec_time = execute_query(query_text, conn)
    performance_without_indexes[query_name] = exec_time
    print(f"{query_name} without indexes: {exec_time:.6f} seconds")

print("\nPerformance without Indexes:")
print(performance_without_indexes)

# Close the connection
conn.close()

# Compare performances
performance_comparison = pd.DataFrame({
    'With Indexes': list(performance_with_indexes.values()),
    'Without Indexes': list(performance_without_indexes.values())
}, index=list(performance_with_indexes.keys()))

performance_comparison['Improvement'] = performance_comparison['Without Indexes'] / performance_comparison['With Indexes']
print("\nPerformance Comparison:")
print(performance_comparison)

rollup with indexes: 13.539606 seconds
drilldown with indexes: 16.598986 seconds
slice with indexes: 4.068030 seconds
dice with indexes: 4.239995 seconds
pivot with indexes: 23.012591 seconds

Performance with Indexes:
{'rollup': 13.539605855941772, 'drilldown': 16.59898591041565, 'slice': 4.06803035736084, 'dice': 4.239995002746582, 'pivot': 23.01259136199951}


DatabaseError: Execution failed on sql '
        SELECT t.year, t.month, SUM(s.quantity_sold) AS total_sales
        FROM sales_fact_with_index s
        JOIN time_dim t ON s.date_id = t.date_id
        GROUP BY t.year, t.month
        ORDER BY t.year, t.month;
    ': no such table: sales_fact_with_index